In [6]:
library(tidyverse)
library(repr)
library(readxl)
source("cleanup.R")
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


## Data Description 
The player.csv data contains information for 196 players, each observation contians 6 variables:
- **experience (character)**: The experience level of players
- **subscription(logical)**: subscribed to a game newsletter or not
- **hashedEmail (character)**：email of the play that has been processed to protect privacy, likely do not have meaningful information to analyse. 
- **played_hours(decimal)**: played time
  - min:0.0 hours
  - 1st Qu.:0.00 hours
  - median: 0.10 hours
  - mean: 5.85 hours
  - 3rd Qu.: 0.60 hours
  - Max.:223.10 hours
- **name(character)**
- **gender(character)**
- **Age(integer)**
  - Min.: 9.0
  - 1st Qu.:17.00
  - median: 19.00
  - mean: 22.75
  - 3rd Qu.: 58.00
  - Max.:58.00
  - Na's:2

In [15]:
players <- read.csv("players.csv")
played_hours_statistic <- players |>
                        select(played_hours)|>
                        summary()

age_statistic <- players |>
                select(Age)|>
                summary()

played_hours_statistic
age_statistic

  played_hours    
 Min.   :  0.000  
 1st Qu.:  0.000  
 Median :  0.100  
 Mean   :  5.846  
 3rd Qu.:  0.600  
 Max.   :223.100  

      Age       
 Min.   : 9.00  
 1st Qu.:17.00  
 Median :19.00  
 Mean   :21.14  
 3rd Qu.:22.75  
 Max.   :58.00  
 NA's   :2      

## Questions
